In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/a-fine-windy-day-hackerearth-ml-challenge/train_data.csv")
test=pd.read_csv("/kaggle/input/a-fine-windy-day-hackerearth-ml-challenge/test_data.csv")
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.info()

In [ ]:
train.describe()

# ****EDA(Exploratory Data Analysis)****

## ****Handling Missing Values****

In [ ]:
train.isna().sum()

train['datetime']=train['datetime'].map(lambda x:str(x).split('-')[0])
test['datetime']=test['datetime'].map(lambda x:str(x).split('-')[0])

In [ ]:
cat_features=['turbine_status','cloud_level']
num_features=[]
for i in train.columns:
    if i not in cat_features and i not in ['tracking_id','datetime']:
        num_features.append(i)
num_features

In [ ]:
for i in num_features:
    train[i].fillna(train[i].median(),inplace=True)
for i in num_features:
    if i!='windmill_generated_power(kW/h)':
        test[i].fillna(test[i].median(),inplace=True)
        
    

In [ ]:
for i in cat_features:
    train[i].fillna(train[i].mode()[0],inplace=True)
    test[i].fillna(test[i].mode()[0],inplace=True)
    

In [ ]:
train.isna().sum()

Handling Outliers

In [ ]:
import matplotlib.pyplot as plt
for i in num_features:
    df=train.copy()
    plt.figure()
    plt.boxplot(train[i])
    plt.xlabel(i)
plt.figure()
plt.boxplot(train['windmill_generated_power(kW/h)'])
plt.show()

import matplotlib.pyplot as plt
for i in num_features:
    df=train.copy()
    plt.figure()
    plt.scatter(train[i],train['windmill_generated_power(kW/h)'])
    plt.xlabel(i)
    plt.ylabel('Target')
    
    

import matplotlib.pyplot as plt
for i in num_features:
    df=train.copy()
    plt.figure()
    plt.hist(train[i],bins=25)
    plt.xlabel(i)
    plt.ylabel('count')
    
    

In [ ]:
for i in cat_features:
    print(test[i].value_counts())
    print()

In [ ]:
mask=train['cloud_level']=='Extremely Low'
mask_test=test['cloud_level']=='Extremely Low'
train.loc[mask,'cloud_level']='Medium'
test.loc[mask_test,'cloud_level']='Medium'

In [ ]:
for i in cat_features:
    train[i]=train[i].astype('category')
    test[i]=test[i].astype('category')

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
for i in cat_features:
    train[i]=lb.fit_transform(train[i])
    test[i]=lb.fit_transform(test[i])

In [ ]:
X=train.drop(['tracking_id','windmill_generated_power(kW/h)','datetime'],axis=1)
Y=train['windmill_generated_power(kW/h)']

In [ ]:
from xgboost import XGBRegressor
model=XGBRegressor(use_label_encoder=True)
model.fit(X,Y)

In [ ]:
t=test.drop(['tracking_id','datetime'],axis=1)
predictions=model.predict(t)

In [ ]:
submission=pd.DataFrame(test['tracking_id'])
submission['datetime']=test['datetime']
submission['windmill_generated_power(kW/h)']=predictions
submission


In [ ]:
submission.to_csv('Sub2.csv',index=False)